# 논문 (git commit log 활용)
## 프로젝트 git 대상 파일 목록을 메뉴별로 mapping

In [1]:
import pandas as pd
import re
import os
import subprocess
from tabulate import tabulate

In [2]:
# 디렉토리에서 include_keywords 포함하고 exclude_keywords 포함하지 않는 파일목록 추출하기
def find_files_keywords(extension, include_keywords, exclude_keywords, directory=r'D:\project\workspace\sanhakin\src\main'):

    matching_files = []

    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_path = os.path.join(root, file)
                with open(file_path, 'r', encoding='utf-8', errors='ignore') as f:
                    for line_number, line in enumerate(f, start=1):
                        include_match = any(re.search(keyword, line) for keyword in include_keywords)
                        exclude_match = any(re.search(keyword, line) for keyword in exclude_keywords)
                        if include_match and not exclude_match and not line.strip().startswith('//'):
                            matching_files.append(f"{file_path} || {line.strip()}")
                            # print(f"Found in: {file_path} || txt: {line.strip()}")
    return matching_files    

def save_txt_file(output_file_path, matching_files):
   with open(output_file_path, 'w', encoding='utf-8') as output_file:
    for rowInfo in matching_files:
        output_file.write(rowInfo + '\n') 

## 1-1. xml파일에서 .do 포함하는 파일 추출하기
### findstr /s  action.*\.do D:\sanhakin\src\main\*.xml > xml_do_list.txt

In [3]:
# xml_do 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
extension = 'xml'
regex_keywords = ['action.*\.do']

matching_files = find_files_keywords(extension, regex_keywords,'')

output_file_path = './txt/xml_do_list.txt'

save_txt_file(output_file_path, matching_files )

data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, "r", encoding="utf-8") as file:
    sch_txt = r'action="/sanhakin'

    for line in file:
        # 각 줄을 ' || '로 분리하여 데이터를 추출합니다.
        parts = line.split(r' || ')
        xml_url = parts[0].replace(r'D:\project\workspace\sanhakin\src\main\webapp', '').replace('\\', '/')

        snd_part = parts[1]
        if sch_txt in snd_part:
            idx = snd_part.find(sch_txt)
            end_idx = snd_part.find('.do', idx)
            if idx != -1 and end_idx != -1:
                do_txt = snd_part[idx + len(sch_txt):end_idx + 3]
                # print("do_txt-->" +  do_txt)
                data.append({'xml_url': xml_url.strip(), 'Do_txt': do_txt.strip()})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/xml_do.csv', encoding="utf-8", index=False)

<>:3: SyntaxWarning: invalid escape sequence '\.'
<>:3: SyntaxWarning: invalid escape sequence '\.'
C:\Users\yjum\AppData\Local\Temp\ipykernel_34648\1595046485.py:3: SyntaxWarning: invalid escape sequence '\.'
  regex_keywords = ['action.*\.do']


## 1-2. xml파일에서 .xml 포함하는 파일 추출하기 (화면에서 다른 메뉴화면 call 하는 경우) //주석으로 된 줄 제외
### findstr /s  \/wqxml.*\.xml D:\sanhakin\src\main\*.xml | findstr /v /i /c:"//" > xml_Inxml.txt 

In [4]:
# xml_do 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
extension = 'xml'
regex_keywords = ['/wqxml/.*\\.xml']

matching_files = find_files_keywords(extension, regex_keywords,'')

output_file_path = './txt/xml_Inxml.txt'

save_txt_file(output_file_path, matching_files )
        
data = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, 'r', encoding='utf-8') as file:

    for line in file:
        # 각 줄을 ' || '로 분리하여 데이터를 추출합니다.
        parts = line.split(r' || ')
        xml_url = parts[0].replace(r'D:\project\workspace\sanhakin\src\main\webapp', '').replace('\\', '/')
        
        #두번째 원소부터 /wqxml/과 .xml 을 포함하는지 체크해서 문자열 추출하기
        snd_part = parts[1]
        if '/wqxml/' in snd_part:
            idx = snd_part.find('/wqxml/')
            end_idx = snd_part.find('.xml', idx)
            
            if idx != -1 and end_idx != -1:
                inXmlPath = snd_part[idx:end_idx + 4]
                # print("inXmlPath String:", inXmlPath)
                # print("xml_path-->" +  xml_path)
                data.append({'xml_url': xml_url.strip(), 'inXml': inXmlPath.strip()})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df = df.drop_duplicates()
# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/xml_Inxml.csv', encoding="utf-8", index=False)

## 2-1. _Controller.java 파일에서
### (1) @RequestMapping .do 포함 텍스트 추출 xml - Controller.java
### (2) @Resource 포함 텍스트 추출           Controller- serviceImpl
### findstr /s  \@RequestMapping\(.*\) D:\sanhakin\src\main\*Controller.java > controller_do.txt

In [5]:
# do_controller 파일을 읽어서 정제 후 do 포함 텍스트와 controller명을 csv 파일로 저장
extension = 'Controller.java'
regex_keywords = ['@RequestMapping\(.*\)', '@Resource\(.*\)']

matching_files = find_files_keywords(extension, regex_keywords, '')

output_file_path = './txt/controller_do.txt'

save_txt_file(output_file_path, matching_files )

# 빈 리스트를 생성하여 데이터 저장 준비
data = []
data2 = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, "r", encoding="utf-8") as file:

    for line in file:
        # parts = re.split(r'@RequestMapping|@Resource', line)
        # 각 줄을 ' || '로 분리하여 데이터를 추출합니다.
        parts = line.split(r' || ')
        controller_url = parts[0].replace(r'D:\project\workspace\sanhakin', '').replace('\\', '/')

        #경로와 파일이름 분리
        controller_path, controller_file = os.path.split(controller_url)
        snd_part = parts[1]
        if '@RequestMapping' in snd_part:
            idx = snd_part.find(",")

            if idx >  -1:
                for text in snd_part.split(","):
                    text = text.strip()
                    do_txt_match = re.search(r'"([^"]*)"', text)
                    if do_txt_match:
                        do_txt = do_txt_match.group(1)
                        data.append({'controller_url': controller_url.strip(), 'controller_file': controller_file.strip(), 'Do_txt': do_txt.strip()})
            else:
                do_txt_match = re.search(r'"([^"]*)"', snd_part)
                if do_txt_match:
                    do_txt = do_txt_match.group(1)
                    data.append({'controller_url': controller_url.strip(), 'controller_file': controller_file.strip(), 'Do_txt': do_txt.strip()})
        
        elif '@Resource' in snd_part:
            resource_match = re.search(r'"([^"]*)"', snd_part)
            if resource_match:
                resource_name = resource_match.group(1)
                data2.append({'controller_url': controller_url.strip(), 'controller_file': controller_file.strip(), 'resource_name': resource_name.strip()})
                
# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)

df = df.drop_duplicates()
df2 = df2.drop_duplicates()

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/controller_do.csv', encoding="utf-8", index=False)
df2.to_csv('./csv/controller_resource.csv', encoding="utf-8", index=False)

<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
<>:3: SyntaxWarning: invalid escape sequence '\('
C:\Users\yjum\AppData\Local\Temp\ipykernel_34648\2156167582.py:3: SyntaxWarning: invalid escape sequence '\('
  regex_keywords = ['@RequestMapping\(.*\)', '@Resource\(.*\)']
C:\Users\yjum\AppData\Local\Temp\ipykernel_34648\2156167582.py:3: SyntaxWarning: invalid escape sequence '\('
  regex_keywords = ['@RequestMapping\(.*\)', '@Resource\(.*\)']


## 2-2. _ServiceImpl.java 파일에서 
### (1) @Service 포함 텍스트 추출 Controller(@Resource)- serviceImpl (@Service)
### (2) @Resource 포함 텍스트 추출 serviceImpl(@Resource)- DAO(@Repository)

In [6]:
# _ServiceImpl 파일을 읽어서 정제 후  @Resource 포함 텍스트와 serviceImpl csv 파일로 저장
extension = 'ServiceImpl.java'
regex_keywords = [r'@Service\(.*\)', r'implements', r'@Resource\(.*\)']

matching_files = find_files_keywords(extension, regex_keywords, '')

output_file_path = './txt/serviceImpl_list.txt'

save_txt_file(output_file_path, matching_files )

# 빈 리스트를 생성하여 데이터 저장 준비
data = []
data2 = []
data3 = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, "r", encoding="utf-8") as file:
    for line in file:
        parts = line.split(r' || ')
        serviceImpl_url = parts[0].replace(r'D:\project\workspace\sanhakin', '').replace('\\', '/')

        #경로와 파일이름 분리
        serviceImpl_path, serviceImpl_file = os.path.split(serviceImpl_url)
        snd_part = parts[1]
        # serviceImpl- Controller
        if '@Service' in snd_part:
            name_match = re.search(r'"([^"]*)"', snd_part)
            if name_match:
                service_name = name_match.group(1)
                data.append({'serviceImpl_url': serviceImpl_url.strip(), 'serviceImpl_file': serviceImpl_file.strip(), 'service_name': service_name.strip()})
        # serviceImpl- Service    
        elif 'implements' in snd_part:    
            name_match = re.search(r'implements\s+([^\s{]+)\s*{', snd_part)
            if name_match:
                implements_name = name_match.group(1)
                data2.append({'serviceImpl_url': serviceImpl_url.strip(), 'serviceImpl_file': serviceImpl_file.strip(), 'implements_name': implements_name.strip()})
                
        # serviceImpl- DAO
        elif '@Resource' in snd_part:
            resource_match = re.search(r'"([^"]*)"', snd_part)
            if resource_match:
                resource_name = resource_match.group(1)
                data3.append({'serviceImpl_url': serviceImpl_url.strip(), 'serviceImpl_file': serviceImpl_file.strip(), 'resource_name': resource_name.strip()})
                
# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)
df3 = pd.DataFrame(data3)

df = df.drop_duplicates()
df2 = df2.drop_duplicates()
df3 = df3.drop_duplicates()

# 데이터프레임을 출력합니다.
df.to_csv('./csv/serviceImpl_service.csv', encoding="utf-8", index=False)
df2.to_csv('./csv/serviceImpl_impl.csv', encoding="utf-8", index=False)
df3.to_csv('./csv/serviceImpl_resource.csv', encoding="utf-8", index=False)

## 3. _DAO.java 파일에서 
### (1) sql_id 텍스트(" "로 묶인 문자열) 추출 
### (2) @Repository 추출  DAO(@Repository) - serviceImpl(@Resource)
### findstr /s /i /r /c:"\".*\"" "D:\sanhakin\src\main\*DAO.java" | findstr /v /i /c:"@SuppressWarnings" | findstr /v /i /c:"@Repository"  > DAO_sql.txt 

In [7]:
# DAO_sql.txt(최초 search 목록에서 eyeCheck로 일부 정제) 파일을 읽어서 정제 후 xml명과 do 포함 텍스트를 csv 파일로 저장
# D:\sanhakin\src\main\java\com\wti\admin\bizDvr\dao\BudgetDvrRqstSttnDAO.java:        return (List<Map>) list("budgetDvrRqstSttnDAO.selectBudgetDvrRqstSttnList", input);
extension = 'DAO.java'
regex_keywords = [r'".*\"']
exclude_keywords = ['@SuppressWarnings']

matching_files = find_files_keywords(extension, regex_keywords, exclude_keywords)

output_file_path = './txt/DAO_sql.txt'

save_txt_file(output_file_path, matching_files )  

data = []
data2 = []

# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, "r", encoding="utf-8") as file:
    for line in file:
        parts = line.split(r' || ')
        DAO_url = parts[0].replace(r'D:\project\workspace\sanhakin', '').replace('\\', '/')

        #경로와 파일이름 분리
        DAO_path, DAO_file = os.path.split(DAO_url)
        snd_part = parts[1]
        
        # serviceImpl- DAO
        if '@Repository' in snd_part:
            name_match = re.search(r'"([^"]*)"', snd_part)
            if name_match:
                repository_name = name_match.group(1)
                data.append({'DAO_url': DAO_url.strip(), 'DAO_file': DAO_file.strip(), 'repository_name': repository_name.strip()})
        # DAO - Sql.xml 
        else:
            sqlID_match = re.search(r'"([^"]*)"', snd_part)
            if sqlID_match:
                sqlID = sqlID_match.group(1)
                data2.append({'DAO_url': DAO_url.strip(), 'DAO_file': DAO_file.strip(), 'sqlID': sqlID.strip()})
  
# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df2 = pd.DataFrame(data2)

df = df.drop_duplicates()
df2 = df2.drop_duplicates()

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/DAO_repository.csv', encoding="utf-8", index=False)
df2.to_csv('./csv/DAO_sqlID.csv', encoding="utf-8", index=False)

## 4. _SQL.xml 파일에서 sqlID 추출 ( "<" ,"id=" ,">" 포함하는 문자열)
### SQLxml_sqlID.txt  ( sqlxml - sqlID )

### findstr /s /r /c:"\<.*\s*id\s*\=\s*\>" D:\sanhakin\src\main\*_SQL.xml  > SQLxml_sqlID.txt

In [8]:
extension = '_SQL.xml'
regex_keywords = [r'<\s*[^>]*\s*id\s*=\s*[^>]*>']
exclude_keywords = []

matching_files = find_files_keywords(extension, regex_keywords, exclude_keywords)

output_file_path = './txt/SQLxml_sqlID.txt'

save_txt_file(output_file_path, matching_files ) 

data = []
reg_txt = r'id\s*=\s*"'
# 텍스트 파일을 한 줄씩 읽어옵니다.
with open(output_file_path, "r", encoding="utf-8") as file:

    for line in file:
        parts = line.split(r' || ')
        SQLxml_url = parts[0].replace(r'D:\project\workspace\sanhakin', '').replace('\\', '/')

        snd_part = parts[1]
        match = re.search(reg_txt + r'(.*?)\"', snd_part)
        if match:
            sqlID = match.group(1)
            data.append({'SQLxml_url': SQLxml_url.strip(), 'sqlID': sqlID.strip()})

# 데이터를 데이터프레임으로 변환합니다.
df = pd.DataFrame(data)
df = df.drop_duplicates()

# 데이터프레임을 출력합니다.
# print(df)
df.to_csv('./csv/SQLxml_sqlID.csv', encoding="utf-8", index=False)

## 5. txt 파일 읽어서 csv 파일로 저장 (데이터 전처리)
### service, serviceImpl, DAO, Controller, _SQL.xml, 화면 xml
#### [ dir /s /b D:\sanhakin\src\main\*service.java > service.txt  ]
#### [ dir /s /b D:\sanhakin\src\main\*serviceImpl.java > serviceImpl.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*DAO.java > DAO.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*Controller.java > Controller.txt ]
#### [ dir /s /b D:\sanhakin\src\main\*_SQl.xml > slqXml.txt ]
#### [ dir /s /b D:\sanhakin\src\main\webapp\*.xml > xm.txt ]

In [9]:
# 파일 목록 가져오기
def find_files_list(extension, file_type, directory=r'D:\project\workspace\sanhakin\src\main'):
    
    matching_files = []
    if file_type == 'xml':
        directory = r'D:\project\workspace\sanhakin\src\main\webapp'
        
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.endswith(extension):
                file_url = os.path.join(root, file)
                file_path, file_name  = os.path.split(file_url)
                file_url = file_url.replace(r"D:\project\workspace\sanhakin", "").replace('\\','/')
                matching_files.append({'file_url': file_url, 'file_name': file_name, 'file_type': file_type})
                # print("file_path: " + file_path +"  extension :" + extension + "  file_type :" +file_type)
    return matching_files  

In [10]:
import csv
extension_arr = ['Service.java', 'ServiceImpl.java', 'DAO.java', 'Controller.java','_SQL.xml', '.xml']
file_tys = ['service', 'Impl', 'dao', 'ctrl', 'sql', 'xml']
output_file_path = './csv/src_files_test.csv'

all_matching_files = []
for i, extension in enumerate(extension_arr):
    matching_files = find_files_list(extension, file_tys[i])
    all_matching_files.extend(matching_files)
    
# Save the data to a CSV file
with open(output_file_path, 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['file_url', 'file_name', 'file_type']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

    # Write the header
    writer.writeheader()
        
    # Write the data
    for row in all_matching_files:
        writer.writerow(row)


In [ ]:
# import csv

# txt_paths = ['./txt/service.txt', './txt/serviceImpl.txt', './txt/DAO.txt', './txt/Controller.txt', './txt/slqXml.txt', './txt/xml.txt']
# csv_paths = ['./csv/service.csv', './csv/serviceImpl.csv', './csv/DAO.csv', './csv/Controller.csv', './csv/slqXml.csv', './csv/xml.csv']
# file_tys = ['service', 'impl', 'dao', 'ctrl', 'sql', 'xml']

# for i, (txt_path, csv_path) in enumerate(zip(txt_paths, csv_paths)):
#     file_ty = file_tys[i]
    
#     # Read the text file and write its contents to a CSV file
#     with open(txt_path, 'r') as txt_file, open(csv_path, 'w', newline='') as csv_file:
#         # Assuming your text file has lines with values separated by spaces
#         reader = csv.reader(txt_file, delimiter=' ')
#         writer = csv.writer(csv_file)
        
#         # header = [prefix + '_path', 'java_file']
#         header = ['file_url', 'file_path', 'file_name', 'file_ty']
#         writer.writerow(header)
        
#         # Write data from the text file to the CSV file
#         for row in reader:
#             # sevice.java 경로 추출하기
#             java_url = row[0].replace("D:\\sanhakin\\", "").replace('\\','/')
#             #경로와 파일이름 분리
#             java_path, java_file  = os.path.split(java_url)
#             java_path = os.path.dirname(java_path)

#             # java_path_file = java_url.split(split_txt)
#             # if len(java_path_file) == 2:
#             #     java_path, java_file = java_path_file
#             # else:
#             #     # Handle the case where the split didn't result in two parts
#             #     java_path, java_file = java_path_file[0], None

#             writer.writerow([java_url, java_path, java_file, file_ty])
            
#     print(f"Conversion from {txt_path} to {csv_path} completed.")

## 5-2 관리대상 파일 하나로 합쳐서 csv 파일로 저장 --> 데이터베이스 wtc_src_file table에 저장

In [11]:
src_files_df = pd.read_csv("./csv/src_files_test.csv", encoding="utf-8") #
src_files_df = src_files_df.drop_duplicates()
src_files_df.fillna('')

src_files_df.sort_values(by=['file_url','file_type'], ascending=False)

# 'file_id' 컬럼 추가하고 규칙에 따라 file_id 값 부여
src_files_df['file_id'] = ['SRC_{:06}'.format(i+1) for i in range(len(src_files_df))]
src_files_df.to_csv('./csv/src_files.csv', encoding="utf-8", index=False) # wtc_src_file table에 저장

In [23]:
src_files_df.head(10)

,file_url,file_name,file_type,file_id
0,/src/main/java/com/wti/admin/bizDvr/service/Bu...,BudgetDvrRqstSttnService.java,service,SRC_000001
1,/src/main/java/com/wti/admin/bizMng/dbMng/serv...,BudgetDvrService.java,service,SRC_000002
2,/src/main/java/com/wti/admin/bizMng/dbMng/serv...,JoinStdnExcelRegService.java,service,SRC_000003
3,/src/main/java/com/wti/admin/bizMng/dbMng/serv...,StdnRegExcelRegInfoService.java,service,SRC_000004
4,/src/main/java/com/wti/admin/bizMng/histMng/se...,HistInqrService.java,service,SRC_000005
5,/src/main/java/com/wti/admin/bizMng/histMng/se...,HistMngExcelRegService.java,service,SRC_000006
6,/src/main/java/com/wti/admin/bizMng/histMng/se...,HistMngService.java,service,SRC_000007
7,/src/main/java/com/wti/admin/bizMng/histMng/se...,HistMngWhlService.java,service,SRC_000008
8,/src/main/java/com/wti/admin/bizMng/hmpyLinkMn...,HmpyLinkMngWhlExcelRegService.java,service,SRC_000009
9,/src/main/java/com/wti/admin/bizMng/hmpyLinkMn...,HmpyLinkMngWhlService.java,service,SRC_000010


## 6. merge 파일별 menu_id 맵핑하기
### menuID_xml   xml_do  controller_do   DAO_sqlID   SQLxml_sqlID

In [12]:
# controller RequestMap 과 xml 화면에서 submition 의 .do 와 맵핑
controller_do = pd.read_csv("./csv/controller_do.csv", encoding="utf-8") #'controller_url', 'Do_txt'
xml_do = pd.read_csv("./csv/xml_do.csv", encoding="utf-8")               # xml_nm', 'Do_txt
controller_xml_df = pd.merge(controller_do, xml_do, on=['Do_txt'], how='inner')
controller_xml_df = controller_xml_df.drop_duplicates(subset=['controller_url','xml_url' ]) 

# WTC_MENU 테이블에서 MENU_URL이 null이 아닌 데이터 추출해서 menu_xml.csv 저장
menuID_xml = pd.read_csv("./csv/menu_xml.csv", encoding="utf-8")
xml_Inxml = pd.read_csv("./csv/xml_Inxml.csv", encoding="utf-8")

#화면 내에서 콜하는 xml 목록에서 메인xml의 menu_id를 찾아서 맵핑
inXml_df = pd.merge(xml_Inxml, menuID_xml, on=['xml_url'], how='inner')[['menu_id','inXml']]
inXml_df.rename(columns={'inXml': 'xml_url'}, inplace = True)

# 1. xml - menu_id
merge_xml_menu_df =  pd.concat([menuID_xml, inXml_df], ignore_index=True)
merge_xml_menu_df = merge_xml_menu_df.drop_duplicates()

# menu_id 에 등록된 화면 xml 과 controller_xml_df의 xml 과 맵핑
# 2. controller 와 xml 파일을 .do로 연결하고 xml의 menu_id를 controller 와 맵핑
merge_xml_ctrl = pd.merge( controller_xml_df, merge_xml_menu_df, on='xml_url', how='left')
merge_xml_ctrl.dropna(subset = ['menu_id'], inplace=True)
ctrl_menu_df = merge_xml_ctrl[['menu_id','controller_url','controller_file']] #549 rows
ctrl_menu_df = ctrl_menu_df.drop_duplicates() #349 rows

In [13]:
# 3. serviceImpl - menu_id
controller_resource = pd.read_csv("./csv/controller_resource.csv", encoding="utf-8")
serviceImpl_service = pd.read_csv("./csv/serviceImpl_service.csv", encoding="utf-8")

serviceImpl_ctrl = pd.merge( controller_resource, serviceImpl_service, left_on='resource_name',right_on='service_name', how = 'inner')
serviceImpl_menu = pd.merge( serviceImpl_ctrl, ctrl_menu_df, on = 'controller_url', how = 'left') 
serviceImpl_menu.dropna(subset = ['menu_id'], inplace=True)
serviceImpl_menu_df = serviceImpl_menu.drop_duplicates(subset=['menu_id','controller_url' ])[['menu_id','serviceImpl_url','serviceImpl_file']] #315 rows


In [14]:
serviceImpl_menu_df.head(10)

,menu_id,serviceImpl_url,serviceImpl_file
0,MENU0000000237,/src/main/java/com/wti/admin/bizDvr/service/Bu...,BudgetDvrRqstSttnServiceImpl.java
1,MENU0000000273,/src/main/java/com/wti/admin/bizDvr/service/Bu...,BudgetDvrRqstSttnServiceImpl.java
2,MENU0000000274,/src/main/java/com/wti/admin/bizDvr/service/Bu...,BudgetDvrRqstSttnServiceImpl.java
3,MENU0000000233,/src/main/java/com/wti/admin/bizMng/dbMng/serv...,BudgetDvrServiceImpl.java
10,MENU0000000055,/src/main/java/com/wti/admin/bizMng/hmpyLinkMn...,HmpyLinkMngWhlServiceImpl.java
11,MENU0000000055,/src/main/java/com/wti/admin/bizMng/hmpyLinkMn...,HmpyLinkMngWhlExcelRegServiceImpl.java
12,MENU0000000132,/src/main/java/com/wti/admin/bizMng/mbrSttn/se...,MbrAdminRoleServiceImpl.java
13,MENU0000000132,/src/main/java/com/wti/admin/bizMng/mbrSttn/se...,MbrAdminRoleDetlServiceImpl.java
15,MENU0000000134,/src/main/java/com/wti/admin/bizMng/mbrSttn/se...,MbrDetlServiceImpl.java
16,MENU0000000131,/src/main/java/com/wti/admin/bizMng/mbrSttn/se...,MbrDetlServiceImpl.java


In [21]:
# 4. serviceImpl - service - menu_id
serviceImpl_impl = pd.read_csv("./csv/serviceImpl_impl.csv", encoding="utf-8")

service_menu_df = pd.merge( serviceImpl_menu_df, serviceImpl_impl, on='serviceImpl_file', how = 'inner')
service_menu_df['implements_name'] = service_menu_df['implements_name'] + '.java'
service_menu_df = service_menu_df[['menu_id','implements_name']] #315 rows

service_menu_df = service_menu_df.rename(columns={'menu_id': 'menu_id', 'implements_name': 'service_file'})
# service_menu_df['service_url'] = None

In [22]:
# 5. DAO(Repository) - serviceImpl(Resource)
serviceImpl_resource = pd.read_csv("./csv/serviceImpl_resource.csv", encoding="utf-8")
DAO_repository = pd.read_csv("./csv/DAO_repository.csv", encoding="utf-8")

# serviceImpl - DAO - menu_id
DAO_serviceImpl = pd.merge( DAO_repository, serviceImpl_resource, left_on='repository_name',right_on='resource_name', how = 'inner')[['DAO_url','DAO_file','serviceImpl_url']]
DAO_menu = pd.merge( DAO_serviceImpl, serviceImpl_menu, on='serviceImpl_url', how = 'left')[['menu_id','DAO_url','DAO_file']]
DAO_menu.dropna(subset = ['menu_id'], inplace=True)
DAO_menu_df = DAO_menu.drop_duplicates(subset=['menu_id','DAO_url' ]) #515 rows

# 6. SQL.xml - menu_id
# DAO 에 있는 sqlId 와 SQL.xml 에 있는 sqlId 맵핑
DAO_sqlID = pd.read_csv("./csv/DAO_sqlID.csv", encoding="utf-8")         # DAO_url', 'slqID
SQLxml_sqlID = pd.read_csv("./csv/SQLxml_sqlID.csv", encoding="utf-8")   # SQLxml_url', 'slqID'

dao_sql_df = pd.merge(DAO_sqlID, SQLxml_sqlID, on=['sqlID'], how='inner')
dao_sql_df = dao_sql_df.drop_duplicates(subset=['DAO_url','SQLxml_url' ])

# sqlxml - DAO - menu_id
sql_menu = pd.merge( DAO_menu_df, dao_sql_df, on='DAO_url', how='inner')[['menu_id','SQLxml_url','DAO_url']]
sql_menu.dropna(subset = ['menu_id'], inplace=True)
sql_menu_df = sql_menu.drop_duplicates(subset=['menu_id','SQLxml_url' ]) #628 rows

In [24]:
# 1. xml
merge_xml_menu_df['file_url'] = '/src/main/webapp' + merge_xml_menu_df['xml_url'] 

xml_fileid = pd.merge( merge_xml_menu_df, src_files_df, on = 'file_url', how = 'inner')[['menu_id','file_id']]
xml_fileid_df = xml_fileid.drop_duplicates(subset=['menu_id','file_id' ])

# 2. controller
ctrl_fileid = pd.merge( ctrl_menu_df, src_files_df, left_on = 'controller_url', right_on = 'file_url',  how = 'inner')[['menu_id','file_id']]
ctrl_fileid_df = ctrl_fileid.drop_duplicates(subset=['menu_id','file_id' ])

# 3. serviceImpl_menu_df
serviceImpl_fileid = pd.merge( serviceImpl_menu_df, src_files_df, left_on = 'serviceImpl_url', right_on = 'file_url',  how = 'left')[['menu_id','file_id']]
serviceImpl_fileid_df = serviceImpl_fileid.drop_duplicates(subset=['menu_id','file_id' ])

# 4. service_menu_df
service_fileid = pd.merge( service_menu_df, src_files_df, left_on = 'service_file', right_on = 'file_name',  how = 'left')[['menu_id','file_id']]
service_fileid_df = service_fileid.drop_duplicates(subset=['menu_id','file_id' ])

# 5. DAO_menu_df
DAO_fileid = pd.merge( DAO_menu_df, src_files_df, left_on = 'DAO_url', right_on = 'file_url',  how = 'left')[['menu_id','file_id']]
DAO_fileid_df = DAO_fileid.drop_duplicates(subset=['menu_id','file_id' ])

# 6. SQL_menu_df
sql_fileid = pd.merge( sql_menu_df, src_files_df, left_on = 'SQLxml_url', right_on = 'file_url',  how = 'left')[['menu_id','file_id']]
sql_fileid_df = sql_fileid.drop_duplicates(subset=['menu_id','file_id' ])

In [27]:
# 타입별 파일 모두 합쳐서 csv 파일로 저장하기
menuID_fileID_df = pd.concat([xml_fileid_df, ctrl_fileid_df, serviceImpl_fileid_df, service_fileid_df, DAO_fileid_df,sql_fileid_df], ignore_index=True)
menuID_fileID_df = menuID_fileID_df.drop_duplicates()
menuID_fileID_df.to_csv('./csv/menuID_fileID.csv', encoding="utf-8", index=False) 

In [28]:
menuID_fileID_df.describe

<bound method NDFrame.describe of              menu_id     file_id
0     MENU0000000178  SRC_002207
1     MENU0000000003  SRC_001875
2     MENU0000000200  SRC_001798
3     MENU0000000048  SRC_001617
4     MENU0000000053  SRC_001608
...              ...         ...
2465  MENU0000000218  SRC_001516
2466  MENU0000000274  SRC_001496
2467  MENU0000000272  SRC_001496
2468  MENU0000000261  SRC_001497
2469  MENU0000000263  SRC_001497

[2470 rows x 2 columns]>

In [ ]:
# print(merge_xml_menu_df.columns)
# print(ctrl_menu_df.columns)
# print(serviceImpl_menu_df.columns)
# print(service_menu_df.columns)
# print(DAO_menu_df.columns)
# print(sql_menu_df.columns)

In [29]:
sql_menu_df.describe()
# print(serviceImpl_menu.info())
# print(merge_xml_ctrl.info())
# print(ctrl_resource_menu.info())
# print(merge_xml_ctrl.groupby(['menu_id','controller_url']).count())
# print(ctrl_resource_menu.groupby(['menu_id','controller_url']).count())

,menu_id,SQLxml_url,DAO_url
count,628,628,628
unique,156,166,158
top,MENU0000000161,/src/main/resources/egovframework/sqlmap/com/c...,/src/main/java/com/wti/usr/aply/dao/SpclhsInst...
freq,23,53,70


## 8-2 메뉴별 파일 합치고  src_files_df 과 merge 해서 file_id mapping하기

In [30]:
# merge_xml_menu_df 424 menu_id  xml_url  
# ctrl_menu_df #1397         file_url  	menu_id
# service_menu_df #1436      file_url  	menu_id
# serviceImpl_menu_df #1544  file_url  	menu_id
# DAO_menu_df #1408          file_url  	menu_id
# sql_menu_df  #1556          DAO_url    DAO_path  SQLxml_url  menu_id

In [32]:
# menuID_java_df = pd.concat([ctrl_menu_df, service_menu_df, serviceImpl_menu_df, DAO_menu_df], ignore_index=True)
# menuID_java_df.descrypt
# menuID_java_df.head()

# menuID_fileURL_df = pd.concat([merge_xml_menu_df, ctrl_menu_df, service_menu_df, serviceImpl_menu_df, DAO_menu_df, sql_menu_df], ignore_index=True)

## [ 최종 ] 메뉴 ID 에 해당하는 파일목록과 commit log 파일 목록 join 해서 찾기
### 파이썬에서 테스트하기

In [33]:
# csv 파일 불러오기. 인코딩은 환경에 따라 변경 필요. 일반적으로 encoding= 'utf-8' 사용
sanhakin_file_df = pd.read_csv("./csv/sanhakin_file.csv", encoding="utf-8") #java_url file_ty   menu_id   
commit_info_df = pd.read_csv("./csv/commit_info.csv", encoding="utf-8") 
commit_file_info_df = pd.read_csv("./csv/commit_file_info.csv", encoding="utf-8")

In [34]:
sanhakin_file_df.head(10)

,menu_id,menu_title,java_url,file_ty
0,MENU0000000237,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/ctrl/Budget...,crtl
1,MENU0000000237,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/service/Bud...,service
2,MENU0000000237,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/service/Bud...,serviceImpl
3,MENU0000000237,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/dao/BudgetD...,DAO
4,MENU0000000237,예산전용 신청현황,src/main/resources/egovframework/sqlmap/com/ad...,sql
5,MENU0000000273,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/ctrl/Budget...,crtl
6,MENU0000000273,예산전용 신청현황,src/main/java/com/wti/admin/cndept/bdgtAplct/c...,crtl
7,MENU0000000273,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/service/Bud...,service
8,MENU0000000273,예산전용 신청현황,src/main/java/com/wti/admin/cndept/bdgtAplct/s...,service
9,MENU0000000273,예산전용 신청현황,src/main/java/com/wti/admin/bizDvr/service/Bud...,serviceImpl


In [35]:
# pd.set_option('display.max_colwidth', None)
# 찾기
# search_term = 'src/main/resources/egovframework/sqlmap/com/usr/aply/idtcag/H3gReqCompInfo_SQL.xml'
# result_df = sanhakin_file_df[sanhakin_file_df['java_url'].str.contains(search_term)]

# Select rows from commit_file_info where 'file_path' matches the condition
menu_selected_files = commit_file_info_df[commit_file_info_df['file_path'].isin(sanhakin_file_df[sanhakin_file_df['menu_id'] == 'MENU0000000160']['java_url'])]
selected_commit = commit_info_df[commit_info_df['commit_id'].isin(menu_selected_files['commit_id'])]
selected_commit.sort_values(by='commit_date', ascending=False)
selected_commit.fillna('')


KeyError: 'commit_date'

In [ ]:
commit_file_info_df[commit_file_info_df['commit_id'] == 'c337f5c43' ]

In [ ]:
merge_df = pd.merge( commit_info_df, commit_file_info_df, on='commit_id', how = 'inner')[['commit_id','commit_date','commit_cls','commit_summ','file_path']]
menu_merge_df = pd.merge( merge_df, sanhakin_file_df, left_on='file_path', right_on = 'java_url',how='inner')[['commit_id','commit_date','menu_id','menu_title','commit_cls','commit_summ','file_path']]
menu_merge_df.head(10)

In [ ]:
# menu_merge_df.groupby(['menu_id','menu_title']).count()
menu_merge_df['commit_date'] = pd.to_datetime(menu_merge_df['commit_date'])
# # Extract year and month into new columns
menu_merge_df['commit_year'] = menu_merge_df['commit_date'].dt.year
menu_merge_df['commit_month'] = menu_merge_df['commit_date'].dt.month

# grouped_df = menu_merge_df.groupby(['commit_year', 'commit_month','menu_id','menu_title']).agg(commit_id_count=('commit_id', 'nunique')).reset_index()
grouped_df = menu_merge_df.groupby(['commit_year', 'commit_month','commit_cls']).agg(commit_id_count=('commit_id', 'nunique')).reset_index()

# menu_merge_df[['cyl','wt']].groupby(['commit_year', 'commit_month','commit_cls']).count()
# Display the result or use it as needed
print(grouped_df)

In [ ]:
## 공부 참고
import pandas as pd
import os

controller_url = "some/path/to/your/directory/your_file.txt"

# Split the path into the directory and the file
controller_path, controller_file = os.path.split(controller_url)

# Get the directory path excluding the last directory name
directory_path_excluding_last = os.path.dirname(controller_path)

# Display the result
print("Directory Path Excluding Last Directory:", directory_path_excluding_last)

# 중복 체크 및 삭제---------------------
# Assuming df is your DataFrame and you want to check duplicates based on columns 'col1' and 'col2'
df = pd.DataFrame({
    'col1': [1, 2, 3, 4, 1],
    'col2': ['A', 'B', 'C', 'D', 'A'],
    'col3': ['X', 'Y', 'Z', 'W', 'X']
})

# Identify duplicate rows based on 'col1' and 'col2'
duplicates = df.duplicated(subset=['col1', 'col2'], keep=False)

# Display the duplicate rows
print(df[duplicates])

# Remove duplicate rows based on 'col1' and 'col2'
df_no_duplicates = df.drop_duplicates(subset=['col1', 'col2'])

# Display the DataFrame without duplicates
print(df_no_duplicates)


## 중복체크
duplicate_values = DAO_sqlID[DAO_sqlID.duplicated(subset='sqlID', keep=False)]
print("DataFrame with Duplicate Values:")
print(duplicate_values)
## null 값 삭제
controller_do.dropna(subset=['controller_file'], inplace=True)
controller_do.describe()